# SSQM v2.0
## Using NBA Shooting Data: Considers shot type, closest defender distance and touch time
## Bin shots by filtering each condition combination and then use that for SSQM

In [1]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import playerdashptshots, leaguedashplayerbiostats

pd.options.mode.chained_assignment = None

data_DIR1 = "../data/box/"
data_DIR2 = "../data/shots/"
csv_export_DIR = "C:/Users/pansr/Documents/repos/csv/"

In [2]:
league = "NBA"
player_dict = get_players_pbp(league=league)
teams_dict, teams_list = get_teams(league="NBA")

In [3]:
season = "2024"
dft = pd.read_parquet(data_DIR1 + f"NBA_Box_P_Cum_Base_"+season+".parquet", columns = ["PLAYER_ID","TEAM_ID"])
df = pd.read_parquet(data_DIR2 + f"{league}_Shots_{season}_All.parquet")
dfd = pd.read_parquet(data_DIR2 + f"{league}_Shots_{season}_overall.parquet", columns = ["PLAYER_ID","FGM","FGA"])
dfd = dfd.sort_values(by = "PLAYER_ID").reset_index(drop=True)

In [4]:
df = df[["PLAYER_ID","PLAYER_NAME","PLAYER_LAST_TEAM_ID","FGM","FGA","FG2M","FG2A","FG3M","FG3A", 'general_range', 'closest_def', 'touch_time']]
df = df.query("general_range != 'Other'")
df_avg = df.groupby(['general_range', 'closest_def', 'touch_time']).sum()
df_avg = df_avg.drop(columns= ["PLAYER_ID","PLAYER_NAME","PLAYER_LAST_TEAM_ID"])
df_avg["xFG2"] = df_avg["FG2M"]/df_avg["FG2A"]
df_avg["xFG3"] = df_avg["FG3M"]/df_avg["FG3A"]
df_avg = df_avg.drop(columns =["FGM","FGA","FG2M","FG2A","FG3M","FG3A"])
df_avg = df_avg.reset_index()

In [5]:
df_avg

,general_range,closest_def,touch_time,xFG2,xFG3
0,Catch and Shoot,0-2 Feet - Very Tight,Touch < 2 Seconds,0.555556,0.295455
1,Catch and Shoot,2-4 Feet - Tight,Touch < 2 Seconds,0.385382,0.303094
2,Catch and Shoot,4-6 Feet - Open,Touch < 2 Seconds,0.464619,0.350387
3,Catch and Shoot,6+ Feet - Wide Open,Touch < 2 Seconds,0.429577,0.391464
4,Less Than 10 ft,0-2 Feet - Very Tight,Touch 2-6 Seconds,0.410825,NaN
5,Less Than 10 ft,0-2 Feet - Very Tight,Touch 6+ Seconds,0.417952,NaN
6,Less Than 10 ft,0-2 Feet - Very Tight,Touch < 2 Seconds,0.541193,NaN
7,Less Than 10 ft,2-4 Feet - Tight,Touch 2-6 Seconds,0.514384,NaN
8,Less Than 10 ft,2-4 Feet - Tight,Touch 6+ Seconds,0.504343,NaN
9,Less Than 10 ft,2-4 Feet - Tight,Touch < 2 Seconds,0.647837,NaN


In [ ]:
dfdasf

In [6]:
shots = pd.merge(df,df_avg,on=['general_range', 'closest_def', 'touch_time'])
shots["FG2_PCT"] = shots["FG2M"]/shots["FG2A"]
shots["FG3_PCT"] = shots["FG3M"]/shots["FG3A"]
shots = shots.replace([np.inf, -np.inf], np.nan)
shots = shots.fillna(0)
shots["PTS2"] =  (2*shots["FG2A"]*shots["FG2_PCT"]).round(2)
shots["PTS3"] =  (3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
shots["PTS"] =  (2*shots["FG2A"]*shots["FG2_PCT"] + 3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
shots["xPTS2"] = (2*shots["FG2A"]*shots["xFG2"]).round(2)
shots["xPTS3"] = (3*shots["FG3A"]*shots["xFG3"]).round(2)
shots["xPTS"] = (2*shots["FG2A"]*shots["xFG2"] + 3*shots["FG3A"]*shots["xFG3"]).round(2)

In [7]:
# shots.query("PLAYER_NAME == 'LeBron James'")

In [8]:
fg = (shots
    .groupby(['PLAYER_ID'])[['FGM', 'FGA', 'PTS', 'xPTS']]
    .agg({'FGM': ["sum"], 'FGA': ["sum"], 'PTS': ["sum"], 'xPTS': ["sum"]}))
fg.columns = ['FGM', 'FGA', 'PTS', 'xPTS']
fg['eFG'] = np.round(fg['PTS']/fg['FGA']/2, 3)
fg['xeFG'] = np.round(fg['xPTS']/fg['FGA']/2, 3)
fg['Shot_Making'] = np.round((fg['PTS'] - fg['xPTS'])/fg['FGA'], 3)

In [9]:
dfd

,PLAYER_ID,FGM,FGA
0,2544,229,468
1,101108,94,220
2,200768,27,75
3,201142,174,332
4,201143,71,158
...,...,...,...
499,1642450,0,2
500,1642461,0,2
501,1642502,0,2
502,1642505,0,3


In [10]:
fg

,FGM,FGA,PTS,xPTS,eFG,xeFG,Shot_Making
PLAYER_ID,,,,,,,
2544,224,455,497.0,480.78,0.546,0.528,0.036
101108,92,218,235.0,219.03,0.539,0.502,0.073
200768,27,75,74.0,78.68,0.493,0.525,-0.062
201142,170,327,387.0,305.72,0.592,0.467,0.249
201143,71,157,185.0,181.16,0.589,0.577,0.024
...,...,...,...,...,...,...,...
1642450,0,2,0.0,2.35,0.000,0.588,-1.175
1642461,0,2,0.0,2.22,0.000,0.555,-1.110
1642502,0,2,0.0,2.25,0.000,0.562,-1.125


In [11]:

fg = fg.drop(columns=['FGM', 'FGA'])
fg = fg.fillna(0)
fg = pd.merge(dfd,fg,on=["PLAYER_ID"])
fg["Points_Added"] = fg["Shot_Making"]* fg["FGA"]
fg["PTS"] = fg["PTS"].astype(int)
fg = fg.reset_index()
fg["Player"] = fg["PLAYER_ID"].map(player_dict)
fg.insert(1,"Player",fg.pop("Player"))
fg = pd.merge(fg,dft,on="PLAYER_ID")
fg["Team"] = fg["TEAM_ID"].map(teams_dict)
fg.insert(2,"Team",fg.pop("Team"))
fg[['Points_Added']] = fg[['Points_Added']].round(1)
fg[['Shot_Making']] = fg[['Shot_Making']].round(3)
fg = fg.drop(columns=["TEAM_ID"])

In [12]:
fg

,index,Player,Team,PLAYER_ID,FGM,FGA,PTS,xPTS,eFG,xeFG,Shot_Making,Points_Added
0,0,LeBron James,Los Angeles Lakers,2544,229,468,497,480.78,0.546,0.528,0.036,16.8
1,1,Chris Paul,San Antonio Spurs,101108,94,220,235,219.03,0.539,0.502,0.073,16.1
2,2,Kyle Lowry,Philadelphia 76ers,200768,27,75,74,78.68,0.493,0.525,-0.062,-4.6
3,3,Kevin Durant,Phoenix Suns,201142,174,332,387,305.72,0.592,0.467,0.249,82.7
4,4,Al Horford,Boston Celtics,201143,71,158,185,181.16,0.589,0.577,0.024,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...
499,499,Daniss Jenkins,Detroit Pistons,1642450,0,2,0,2.35,0.000,0.588,-1.175,-2.4
500,500,Spencer Jones,Denver Nuggets,1642461,0,2,0,2.22,0.000,0.555,-1.110,-2.2
501,501,Malevy Leons,Oklahoma City Thunder,1642502,0,2,0,2.25,0.000,0.562,-1.125,-2.2
502,502,Alex Ducas,Oklahoma City Thunder,1642505,0,3,0,3.11,0.000,0.518,-1.037,-3.1


In [13]:
df = fg.iloc[:,1:].query("PTS > 100").nlargest(10,columns="Shot_Making").reset_index(drop=True).drop(columns="PLAYER_ID")
df.index += 1
print(df.to_markdown())

|    | Player           | Team             |   FGM |   FGA |   PTS |   xPTS |   eFG |   xeFG |   Shot_Making |   Points_Added |
|---:|:-----------------|:-----------------|------:|------:|------:|-------:|------:|-------:|--------------:|---------------:|
|  1 | Garrison Mathews | Atlanta Hawks    |    52 |   110 |   150 | 112.62 | 0.682 |  0.512 |         0.34  |           37.4 |
|  2 | AJ Green         | Milwaukee Bucks  |    70 |   155 |   202 | 157.76 | 0.652 |  0.509 |         0.285 |           44.2 |
|  3 | Larry Nance Jr.  | Atlanta Hawks    |    46 |    79 |   113 |  92.3  | 0.724 |  0.592 |         0.265 |           20.9 |
|  4 | Kevin Durant     | Phoenix Suns     |   174 |   332 |   387 | 305.72 | 0.592 |  0.467 |         0.249 |           82.7 |
|  5 | Daniel Gafford   | Dallas Mavericks |   143 |   195 |   286 | 240.12 | 0.733 |  0.616 |         0.235 |           45.8 |
|  6 | Payton Pritchard | Boston Celtics   |   155 |   325 |   418 | 344.84 | 0.643 |  0.531 |         0

In [14]:
# fg.sort_values(by="Points_Added")

In [15]:
# kjh

In [21]:
df_e = fg.iloc[:,1:].query("PTS > 100").nlargest(10,columns="Points_Added").reset_index(drop=True)
df_e = df_e.drop(columns=["PLAYER_ID"])
df_e.index += 1
df_e

,Player,Team,FGM,FGA,PTS,xPTS,eFG,xeFG,Shot_Making,Points_Added
1,Giannis Antetokounmpo,Milwaukee Bucks,307,501,614,529.04,0.615,0.530,0.170,85.2
2,Kevin Durant,Phoenix Suns,174,332,387,305.72,0.592,0.467,0.249,82.7
3,Norman Powell,LA Clippers,192,388,472,391.20,0.608,0.504,0.208,80.7
4,Shai Gilgeous-Alexander,Oklahoma City Thunder,302,592,664,585.99,0.562,0.496,0.132,78.1
5,Zach LaVine,Chicago Bulls,201,404,481,406.75,0.606,0.512,0.187,75.5
6,Payton Pritchard,Boston Celtics,155,325,418,344.84,0.643,0.531,0.225,73.1
7,Tyler Herro,Miami Heat,224,475,556,484.08,0.586,0.511,0.152,72.2
8,Jalen Brunson,New York Knicks,241,494,556,485.36,0.569,0.496,0.144,71.1
9,Darius Garland,Cleveland Cavaliers,211,428,509,440.21,0.595,0.514,0.161,68.9
10,Nikola Vucevic,Chicago Bulls,241,417,540,475.43,0.651,0.573,0.156,65.1


In [22]:
df_e = fg.iloc[:,1:].query("PTS > 100").nlargest(10,columns="Shot_Making").reset_index(drop=True)
df_e = df_e.drop(columns=["PLAYER_ID"])
df_e.index += 1
df_e

,Player,Team,FGM,FGA,PTS,xPTS,eFG,xeFG,Shot_Making,Points_Added
1,Garrison Mathews,Atlanta Hawks,52,110,150,112.62,0.682,0.512,0.340,37.4
2,AJ Green,Milwaukee Bucks,70,155,202,157.76,0.652,0.509,0.285,44.2
3,Larry Nance Jr.,Atlanta Hawks,46,79,113,92.30,0.724,0.592,0.265,20.9
4,Kevin Durant,Phoenix Suns,174,332,387,305.72,0.592,0.467,0.249,82.7
5,Daniel Gafford,Dallas Mavericks,143,195,286,240.12,0.733,0.616,0.235,45.8
6,Payton Pritchard,Boston Celtics,155,325,418,344.84,0.643,0.531,0.225,73.1
7,Norman Powell,LA Clippers,192,388,472,391.20,0.608,0.504,0.208,80.7
8,Josh Hart,New York Knicks,151,258,340,288.80,0.659,0.560,0.198,51.1
9,Zach LaVine,Chicago Bulls,201,404,481,406.75,0.606,0.512,0.187,75.5
10,Jamison Battle,Toronto Raptors,57,118,152,130.04,0.644,0.551,0.186,21.9


In [17]:
%reload_ext rpy2.ipython

c:\Users\pansr\AppData\Local\pypoetry\Cache\virtualenvs\nba-ub9Z_EQq-py3.11\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.


In [18]:
%%R -i df_e
library(tidyverse)
library(gt)
df <- df_e
df %>% 
  gt()%>%
  tab_header(
    title = md("**Best Volume Shot Makers 2023-24**"),
    subtitle = "Based on SSQM v2.0 | Shot Making: Points per shot (PPS) above league average" 
    ) %>%
    data_color(columns = c(eFG,xeFG,Shot_Making,Points_Added), palette = "YlGn",method="quantile",quantiles=9) %>%
    cols_align(align = "center",columns = c(Shot_Making,Points_Added))  %>%
    cols_label(
      Shot_Making = "Shot Making", eFG = "eFG%", xeFG = "xeFG%", Points_Added = "Points Added"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "eFG%: Effective Field Goal % | xeFG%: Expected eFG% | xPTS: Expected Points")  %>% 
    tab_source_note(
    source_note = "Simple Shot Quality Model: SSQM v2.0 is based on shot type, defender distance & touch time" ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/player_leaders/ssqm2_points_added.png",size=5) 

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.0     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Error in webshot2::webshot(url = paste0("file:///", tempfile_), file = filename,  : 
  unused argument (size = 5)
In addition: Warning messages:
1: package 'ggplot2' was built under R version 4.3.3 
2: package 'tidyr' was built under R version 4.3.3 
3: package 'readr' was built under R version 4.3.3 
4: Skewed data means we can only allocate 9 unique colours not the 9 requested
Error in webshot2::webshot(url = paste0("file:///", tempfile_), file = filename,  : 
  unused 

RInterpreterError: Failed to parse and evaluate line 'library(tidyverse)\nlibrary(gt)\ndf <- df_e\ndf %>% \n  gt()%>%\n  tab_header(\n    title = md("**Best Volume Shot Makers 2023-24**"),\n    subtitle = "Based on SSQM v2.0 | Shot Making: Points per shot (PPS) above league average" \n    ) %>%\n    data_color(columns = c(eFG,xeFG,Shot_Making,Points_Added), palette = "YlGn",method="quantile",quantiles=9) %>%\n    cols_align(align = "center",columns = c(Shot_Making,Points_Added))  %>%\n    cols_label(\n      Shot_Making = "Shot Making", eFG = "eFG%", xeFG = "xeFG%", Points_Added = "Points Added"\n    ) %>%\n    tab_options(\n        table.background.color = "floralwhite",\n        column_labels.font.size = 12,\n        column_labels.font.weight = \'bold\',\n        row_group.font.weight = \'bold\',\n        row_group.background.color = "#E5E1D8",\n        table.font.size = 10,\n        heading.title.font.size = 20,\n        heading.subtitle.font.size = 12.5,\n        table.font.names = 

In [ ]:
df_e = fg.iloc[:,1:].query("PTS > 100").nsmallest(10,columns="Points_Added").reset_index(drop=True)
df_e = df_e.drop(columns=["PLAYER_ID"])
df_e.index += 1

In [ ]:
%%R -i df_e
library(tidyverse)
library(gt)
df <- df_e
df %>% 
  gt()%>%
  tab_header(
    title = md("**Worst Volume Shot Makers 2023-24**"),
    subtitle = "Based on SSQM v2.0 | Shot Making: Points per shot (PPS) above league average" 
    ) %>%
    data_color(columns = c(eFG,xeFG,Shot_Making,Points_Added), palette = "OrRd",method="quantile",quantiles=9, reverse=TRUE) %>%
    cols_align(align = "center",columns = c(Shot_Making,Points_Added))  %>%
    cols_label(
      Shot_Making = "Shot Making", eFG = "eFG%", xeFG = "xeFG%", Points_Added = "Points Added"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "eFG%: Effective Field Goal % | xeFG%: Expected eFG% | xPTS: Expected Points")  %>% 
    tab_source_note(
    source_note = "Simple Shot Quality Model: SSQM v2.0 is based on shot type, defender distance & touch time" ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/player_leaders/ssqm2_points_added_worst.png",size=5) 

In [ ]:
df_e = fg.iloc[:,1:].query("PTS > 500").nsmallest(10,columns="xeFG").reset_index(drop=True)
df_e = df_e.drop(columns=["PLAYER_ID"])
df_e.index += 1
df_e

In [ ]:
%%R -i df_e
library(tidyverse)
library(gt)
df <- df_e
df %>% 
  gt()%>%
  tab_header(
    title = md("**Toughest Shot Takers 2023-24**"),
    subtitle = "Based on SSQM v2.0 | Among Players Scoring at least 500 Pts " 
    ) %>%
    data_color(columns = c(eFG,xeFG), palette = "OrRd",method="quantile",quantiles=9, reverse=TRUE) %>%
    data_color(columns = c(Shot_Making,Points_Added), palette = "YlGn",method="quantile",quantiles=9) %>%
    cols_align(align = "center",columns = c(Shot_Making,Points_Added))  %>%
    cols_label(
      Shot_Making = "Shot Making", eFG = "eFG%", xeFG = "xeFG%", Points_Added = "Points Added"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "Shot Making: Points per shot (PPS) above league average | eFG%: Effective Field Goal % | xeFG%: Expected eFG% | xPTS: Expected Points")  %>% 
    tab_source_note(
    source_note = "Simple Shot Quality Model: SSQM v2.0 is based on shot type, defender distance & touch time" ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/player_leaders/ssqm2_toughest.png",size=5) 

In [ ]:
df_e = fg.iloc[:,1:].query("PTS > 100").query("Team == 'New Orleans Pelicans'").sort_values("Points_Added",ascending=False).reset_index(drop=True)
df_e = df_e.drop(columns=["PLAYER_ID","Team"])
df_e.index += 1

In [ ]:
%%R -i df_e
library(tidyverse)
library(gt)
df <- df_e
df %>% 
  gt()%>%
  tab_header(
    title = md("**New Orleans Pelicans Shot Making 2023-24**"),
    subtitle = "Based on SSQM v2.0 | Shot Making: Points per shot (PPS) above league average" 
    ) %>%
    data_color(columns = c(eFG,xeFG,Shot_Making,Points_Added), palette = "PiYG") %>%
    cols_align(align = "center",columns = c(Shot_Making,Points_Added))  %>%
    cols_label(
      Shot_Making = "Shot Making", eFG = "eFG%", xeFG = "xeFG%", Points_Added = "Points Added"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "Shot Making: Points per shot (PPS) above league average")  %>% 
     tab_source_note(
    source_note = "eFG%: Effective Field Goal % | xeFG%: Expected eFG% | xPTS: Expected Points")  %>% 
    tab_source_note(
    source_note = "Simple Shot Quality Model: SSQM v2.0 is based on shot type, defender distance & touch time" ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/player_leaders/pelicans_points_added.png",size=5) 

In [ ]:
xcvcxvx

In [ ]:
export_DIR = "../../repos/csv/"

In [ ]:
# df_e.to_csv(export_DIR + "NBA_Shot_Quality.csv")

In [ ]:
df_e.query("Player == 'Davion Mitchell'")

In [ ]:
df_e.sort_values("PTS")

In [ ]:
df_e.to_csv("NBA_Shot_Quality_V2.csv")

# To Website

In [ ]:
shots = pd.merge(df,df_avg,on=['general_range', 'closest_def', 'touch_time'])
shots["FG2_PCT"] = shots["FG2M"]/shots["FG2A"]
shots["FG3_PCT"] = shots["FG3M"]/shots["FG3A"]
shots = shots.replace([np.inf, -np.inf], np.nan)
shots = shots.fillna(0)
shots["PTS2"] =  (2*shots["FG2A"]*shots["FG2_PCT"]).round(2)
shots["PTS3"] =  (3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
shots["PTS"] =  (2*shots["FG2A"]*shots["FG2_PCT"] + 3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
shots["xPTS2"] = (2*shots["FG2A"]*shots["xFG2"]).round(2)
shots["xPTS3"] = (3*shots["FG3A"]*shots["xFG3"]).round(2)
shots["xPTS"] = (2*shots["FG2A"]*shots["xFG2"] + 3*shots["FG3A"]*shots["xFG3"]).round(2)

In [ ]:
fg = (
    shots
    .groupby(['PLAYER_ID'])[['FG2M', 'FG2A', 'FG3M', 'FG3A', 'FGM', 'FGA', 'PTS2', 'PTS3', 'PTS', 'xPTS2', 'xPTS3', 'xPTS']]
    .sum()
)
fg.columns = ['FG2M', 'FG2A', 'FG3M', 'FG3A','FGM', 'FGA', 'PTS2', 'PTS3', 'PTS', 'xPTS2', 'xPTS3', 'xPTS']
fg['FG2_PCT'] = np.round(fg['FG2M']/fg['FG2A'], 3)
fg['FG3_PCT'] = np.round(fg['FG3M']/fg['FG3A'], 3)
fg['eFG'] = np.round(fg['PTS']/fg['FGA']/2, 3)
fg['xeFG'] = np.round(fg['xPTS']/fg['FGA']/2, 3)
fg['Shot_Making2'] = np.round((fg['PTS2'] - fg['xPTS2'])/fg['FG2A'], 3)
fg['Shot_Making3'] = np.round((fg['PTS3'] - fg['xPTS3'])/fg['FG3A'], 3)
fg['Shot_Making'] = np.round((fg['PTS'] - fg['xPTS'])/fg['FGA'], 3)
fg = fg.drop(columns=['FGM', 'FGA'])
fg = fg.fillna(0)
fg = pd.merge(dfd,fg,on=["PLAYER_ID"])
fg["Points_Added2"] = fg['PTS2'] - fg['xPTS2']
fg["Points_Added3"] = fg['PTS3'] - fg['xPTS3']
fg["Points_Added"] = fg['PTS'] - fg['xPTS']
# fg["Points_Added"] = fg["Shot_Making"]* fg["FGA"]
fg["PTS"] = fg["PTS"].astype(int)
fg = fg.reset_index()
fg["Player"] = fg["PLAYER_ID"].map(player_dict)
fg.insert(1,"Player",fg.pop("Player"))
fg = pd.merge(fg,dft,on="PLAYER_ID")
fg["Team"] = fg["TEAM_ID"].map(teams_dict)
fg.insert(2,"Team",fg.pop("Team"))
fg[['Points_Added']] = fg[['Points_Added']].round(1)
fg[['Shot_Making']] = fg[['Shot_Making']].round(3)
fg = fg.drop(columns=["TEAM_ID"])
fg

In [ ]:
fg1 = fg.drop(columns=['index','FGM', 'FGA', 'FG2M', 'FG2A',
       'FG3M', 'FG3A',])
fg1 = fg1[['Player', 'Team', 'PLAYER_ID', 
            'PTS2', 'xPTS2', 'FG2_PCT', 'Shot_Making2', 'Points_Added2', 
            'PTS3', 'xPTS3', 'FG3_PCT', 'Shot_Making3', 'Points_Added3', 
            'PTS', 'xPTS', 'eFG', 'xeFG', 'Shot_Making', 'Points_Added'
       ]]
fg1[['xPTS2','Points_Added2','xPTS3','Points_Added3',]] = fg1[['xPTS2','Points_Added2','xPTS3','Points_Added3',]].round(2)
fg1[['xPTS','Points_Added',]] = fg1[['xPTS','Points_Added']].round(2)
fg1 = fg1.query("PTS >= 100").reset_index(drop=True)
fg1.to_csv(csv_export_DIR + "NBA_Shot_Quality_V2.csv")

In [ ]:
fg1.iloc[:30]

In [ ]:
csv_export_DIR